### Importing libraries

In [7]:
import os
import sys
import numpy as np
import rasterio
import torch
import shutil
import random
import pathlib
from shapely.geometry import LineString, Polygon
from scipy.spatial import ConvexHull
from rasterio.features import rasterize
from shapely.geometry import shape, box
from shapely.validation import make_valid
import geopandas as gpd
from tqdm import tqdm
import json
import cv2
from rasterio.windows import Window
from rasterio.plot import show
import matplotlib.pyplot as plt

In [2]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
CUDA version: 12.1
GPU name: NVIDIA GeForce GTX 1650 Ti


In [3]:
PROJECT_DIR = "C://Users//Amir//Desktop//Thesis"

### Slicing 6 Shymkent rasters to 1280

In [4]:
def slice_rasters(tiff_files, output_dir, tile_size=1280):
    os.makedirs(output_dir, exist_ok=True)

    for input_raster_path in tiff_files:
        base_name = os.path.splitext(os.path.basename(input_raster_path))[0]
        metadata = {
            "original_file": input_raster_path,
            "tiles": []
        }

        with rasterio.open(input_raster_path) as src:
            width, height = src.width, src.height
            cols = (width + tile_size - 1) // tile_size
            rows = (height + tile_size - 1) // tile_size

            for i in range(rows):
                for j in range(cols):
                    xmin = j * tile_size
                    ymin = i * tile_size
                    xmax = min(xmin + tile_size, width)
                    ymax = min(ymin + tile_size, height)

                    # Determine if we need padding
                    pad_right = tile_size - (xmax - xmin)
                    pad_bottom = tile_size - (ymax - ymin)

                    window = Window(xmin, ymin, xmax - xmin, ymax - ymin)
                    transform = rasterio.windows.transform(window, src.transform)
                    # Read the data and pad if necessary
                    tile_data = src.read(window=window)
                    if pad_right > 0 or pad_bottom > 0:
                        padded_data = np.zeros((src.count, tile_size, tile_size), dtype=src.dtypes[0])
                        padded_data[:, :ymax-ymin, :xmax-xmin] = tile_data
                        tile_data = padded_data

                    tile_id = f"{base_name}_tile_{i}_{j}"
                    tile_path = os.path.join(output_dir, f"{tile_id}.tif")

                    with rasterio.open(
                        tile_path,
                        'w',
                        driver='GTiff',
                        height=tile_size,
                        width=tile_size,
                        count=src.count,
                        dtype=src.dtypes[0],
                        crs=src.crs,
                        transform=transform
                    ) as dst:
                        dst.write(tile_data)

                    metadata["tiles"].append({
                        "tile_id": tile_id,
                        "tile_row": i,
                        "tile_col": j,
                        "x_offset": xmin,
                        "y_offset": ymin,
                        "needs_padding": (pad_right > 0 or pad_bottom > 0),
                        "original_width": xmax - xmin,
                        "original_height": ymax - ymin,
                        "crs": src.crs.to_string(),
                        "transform": list(transform)
                    })

        metadata_path = os.path.join(output_dir, f"{base_name}_metadata.json")
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)

        print(f"Sliced {input_raster_path} into {rows * cols}.")

tiff_files = [
    r"C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_9_27.tif",
    r"C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_9_31.tif",
    r"C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_10_26.tif",
    r"C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_13_34.tif",
    r"C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_13_35.tif",
    r"C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_14_26.tif",
    r"C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_15_28.tif",
    r"C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_16_28.tif",
    r"C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_17_28.tif",   
]

slice_rasters(tiff_files, PROJECT_DIR + "//raster_tiles_Shymkent")

Sliced C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_9_27.tif into 256.
Sliced C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_9_31.tif into 256.
Sliced C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_10_26.tif into 256.
Sliced C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_13_34.tif into 256.
Sliced C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_13_35.tif into 256.
Sliced C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_14_26.tif into 256.
Sliced C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_15_28.tif into 256.
Sliced C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_16_28.tif into 256.
Sliced C:/Users/Amir/Desktop/Thesis/Tiff_files/Shymkent_5cm_17_28.tif into 256.


### Clips vectors based on metadata + counts tiles on diff vectors - Shymkent

In [6]:
def clip_vectors_multiple(vector_paths, raster_tiles_dir, output_dir, target_crs="EPSG:32643"):
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize tile statistics
    stats = {
        'total_tiles': 0,
        'tiles_with_buildings': 0,
        'tiles_with_roads': 0,
        'tiles_with_both': 0,
        'empty_tiles': 0,
        'total_buildings': 0,
        'total_roads': 0,
        'avg_buildings_per_tile': 0,
        'avg_roads_per_tile': 0
    }
    
    # Track vector counts per tile
    buildings_counts = []
    roads_counts = []

    for vector_path, class_id in vector_paths:
        print(f"\nProcessing class {class_id}...")
        class_name = 'builds' if class_id == 0 else 'street'
        try:
            vectors = gpd.read_file(vector_path, encoding='cp1251')
            vectors['class'] = class_id  # Add a column for class

            # Reproject to the target CRS
            if vectors.crs != target_crs:
                vectors = vectors.to_crs(target_crs)

            # Fix geometries using make_valid (more robust than buffer(0))
            vectors['geometry'] = vectors['geometry'].apply(
                lambda geom: make_valid(geom) if geom and not geom.is_empty else None
            )
            
            # Simplify only polygons (not roads) to preserve connectivity
            if class_name == 'builds':
                vectors['geometry'] = vectors['geometry'].simplify(tolerance=0.5)

            # Remove any null geometries
            vectors = vectors[vectors.geometry.notnull()]

            # Load raster metadata (CRS, bounds)
            metadata_files = [f for f in os.listdir(raster_tiles_dir) if f.endswith('_metadata.json')]

            for meta_file in tqdm(metadata_files, desc=f"Processing tiles for class {class_name}"):
                with open(os.path.join(raster_tiles_dir, meta_file)) as f:
                    meta = json.load(f)

                for tile in meta["tiles"]:
                    tile_id = tile['tile_id']
                    tile_tif_path = os.path.join(raster_tiles_dir, f"{tile_id}.tif")

                    # Set output paths based on class
                    if class_name == 'builds':
                        output_path = os.path.join(output_dir, f"{tile_id}_builds_vectors.shp")
                    else:
                        output_path = os.path.join(output_dir, f"{tile_id}_street_vectors.geojson")

                    # Skip if the output file already exists
                    if os.path.exists(output_path):
                        continue

                    if not os.path.exists(tile_tif_path):
                        print(f"Warning: Tile image not found for {tile_id}")
                        continue

                    try:
                        with rasterio.open(tile_tif_path) as src:
                            # Get tile bounds in CRS coordinates
                            bounds = src.bounds
                            tile_crs = src.crs

                        # Reproject vectors to tile CRS if needed
                        if vectors.crs != tile_crs:
                            vectors_proj = vectors.to_crs(tile_crs)
                        else:
                            vectors_proj = vectors

                        # Create bounding box for the tile
                        tile_bounds_geom = box(*bounds)

                        # Get intersecting features
                        mask = vectors_proj.intersects(tile_bounds_geom)
                        vectors_in_tile = vectors_proj[mask].copy()

                        # For roads, we need to preserve the full geometry for reconstruction
                        if class_name == 'street':
                            # Store original geometry as WKT before clipping
                            vectors_in_tile['original_geom_wkt'] = vectors_in_tile['geometry'].to_wkt()
                            
                            # Clip to tile bounds
                            vectors_in_tile['geometry'] = vectors_in_tile.intersection(tile_bounds_geom)
                        else:
                            vectors_in_tile['geometry'] = vectors_in_tile.intersection(tile_bounds_geom)

                        # Filter empty geometries
                        vectors_in_tile = vectors_in_tile[~vectors_in_tile.is_empty]
                        vectors_in_tile['geometry'] = vectors_in_tile['geometry'].apply(
                            lambda geom: make_valid(geom) if geom and not geom.is_empty else None
                        )
                        vectors_in_tile = vectors_in_tile[vectors_in_tile.geometry.notnull()]

                        if not vectors_in_tile.empty:
                            # Update statistics
                            count = len(vectors_in_tile)
                            if class_name == 'builds':
                                buildings_counts.append(count)
                                stats['total_buildings'] += count
                            else:
                                roads_counts.append(count)
                                stats['total_roads'] += count
                                
                            # Save the clipped vectors
                            if class_name == 'street':
                                vectors_in_tile.to_file(output_path, driver='GeoJSON')
                            else:
                                vectors_in_tile.to_file(output_path)

                    except Exception as e:
                        print(f"\nError processing tile {tile_id} for class {class_name}: {str(e)}")
                        continue

        except Exception as e:
            print(f"\nError processing class {class_name}: {str(e)}")
            continue
    
    # Calculate final statistics
    stats['total_tiles'] = len(buildings_counts) + len(roads_counts)
    
    if buildings_counts:
        stats['tiles_with_buildings'] = len(buildings_counts)
        stats['avg_buildings_per_tile'] = sum(buildings_counts) / len(buildings_counts)
    
    if roads_counts:
        stats['tiles_with_roads'] = len(roads_counts)
        stats['avg_roads_per_tile'] = sum(roads_counts) / len(roads_counts)
    
    # Print summary
    print("\n=== Processing Summary ===")
    print(f"Total tiles processed: {stats['total_tiles']}")
    print(f"Tiles with buildings: {stats['tiles_with_buildings']}")
    print(f"Tiles with roads: {stats['tiles_with_roads']}")
    print(f"Average buildings per tile: {stats['avg_buildings_per_tile']:.1f}")
    print(f"Average roads per tile: {stats['avg_roads_per_tile']:.1f}")
    print(f"Total buildings processed: {stats['total_buildings']}")
    print(f"Total roads processed: {stats['total_roads']}")
    return stats

vector_paths = [
    (PROJECT_DIR + "//Shymkent_vectors//builds.shp", 0),
    (PROJECT_DIR + "//Shymkent_vectors//street.shp", 1)
]
clip_vectors_multiple(vector_paths, PROJECT_DIR+"//raster_tiles_Shymkent", PROJECT_DIR+"//vector_tiles_Shymkent")


Processing class 0...


c:\Users\Amir\miniconda3\envs\jupyter_gpu\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: One or several characters couldn't be converted correctly from cp1251 to UTF-8. This warning will not be emitted anymore
  return ogr_read(
Processing tiles for class builds:   0%|          | 0/9 [00:00<?, ?it/s]c:\Users\Amir\miniconda3\envs\jupyter_gpu\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 'Р РµСЃРїСѓР±Р»РёРєР° РљР°Р·Р°С…СЃС‚Р°РЅ, РіРѕСЂРѕРґ РЁС‹РјРєРµРЅС‚, СЂР°Р№РѕРЅ РљР°СЂР°С‚Р°Сѓ, РјРёРєСЂРѕСЂР°Р№РѕРЅ РђСЃР°СЂ, СѓР»РёС†Р° Р–Р°РЅР°РєСѓСЂС‹Р»С‹СЃ, РґРѕРј 652' of field full_path_ has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(
c:\Users\Amir\miniconda3\envs\jupyter_gpu\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 'Р РµСЃРїСѓР±Р»РёРєР° РљР°Р·Р°С…СЃС‚Р°РЅ, РіРѕСЂРѕРґ РЁС‹РјРєРµРЅС‚, РјРёРєСЂРѕСЂР°Р№РѕРЅ РђСЃР°СЂ, СѓР»РёС†Р° РўРѕР»С‚Р°Р№, РґРѕРј 52' of field full_path_ has been truncated to 254 ch


Processing class 1...


c:\Users\Amir\miniconda3\envs\jupyter_gpu\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: One or several characters couldn't be converted correctly from cp1251 to UTF-8. This warning will not be emitted anymore
  return ogr_read(
Processing tiles for class street: 100%|██████████| 9/9 [02:36<00:00, 17.34s/it]


=== Processing Summary ===
Total tiles processed: 3115
Tiles with buildings: 1801
Tiles with roads: 1314
Average buildings per tile: 7.5
Average roads per tile: 1.3
Total buildings processed: 13483
Total roads processed: 1752


{'total_tiles': 3115,
 'tiles_with_buildings': 1801,
 'tiles_with_roads': 1314,
 'tiles_with_both': 0,
 'empty_tiles': 0,
 'total_buildings': 13483,
 'total_roads': 1752,
 'avg_buildings_per_tile': 7.486396446418656,
 'avg_roads_per_tile': 1.3333333333333333}

### Visualization of the raster and vector tile

In [ ]:
def visualize_tile(raster_tile_path, vector_tile_path):

    # Create figure
    fig, ax = plt.subplots(figsize=(6, 6))

    # 1. Plot the raster tile
    with rasterio.open(raster_tile_path) as src:
        show(src, ax=ax, cmap='gray')

    # 2. Plot the vector tile (if exists)
    try:
        vectors = gpd.read_file(vector_tile_path)
        if not vectors.empty:
            vectors.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=1)
            print(f"Found {len(vectors)} vector features in tile.")
        else:
            print("No vector features in this tile.")
    except Exception as e:
        print(f"Vector tile not found or error: {e}")

    # Customize plot
    plt.title(f"Raster & Vector Tile")
    plt.axis('on')
    plt.show()

# Example usageC://Users//Amir//Desktop//Thesis//yolo_dataset_overlap256//raster_tiles_overlap256\Pavlodar_AFS_2019_13_7_tile_0_0.tif
# Pavlodar_AFS_2019_7_3_tile_19_14
visualize_tile(
    raster_tile_path=PROJECT_DIR+"//raster_tiles_Shymkent//Shymkent_5cm_13_35_tile_0_1.tif",
    vector_tile_path=PROJECT_DIR+"//vector_tiles_Shymkent//Shymkent_5cm_13_35_tile_0_1_street_vectors.geojson"
)

### Converting TIFF to PNG and shapefiles and GeoJSON (street lines) to YOLO-Seg TXT

In [ ]:
def convert_tiles_to_png_and_yolo_seg(raster_tiles_dir, vector_tiles_dir, output_img_dir, output_lbl_dir, class_map, save_empty_labels=True):
    """
    Convert all raster tiles to PNG and generate YOLO-Seg annotations from both street (GeoJSON) and builds (SHP) vector data.
    Output format: <class-id> <x1> <y1> <x2> <y2> ... <xn> <yn> (normalized coordinates)
    """
    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_lbl_dir, exist_ok=True)

    # Loop over all raster tiles
    tiff_files = [f for f in os.listdir(raster_tiles_dir) if f.endswith('.tif')]

    for tif_file in tiff_files:
        tile_id = tif_file.replace('.tif', '')
        tif_path = os.path.join(raster_tiles_dir, tif_file)
        png_path = os.path.join(output_img_dir, f"{tile_id}.png")
        txt_path = os.path.join(output_lbl_dir, f"{tile_id}.txt")

        # Convert TIFF to PNG
        with rasterio.open(tif_path) as src:
            img = src.read([1, 2, 3]) if src.count >= 3 else src.read()
            img = reshape_as_image(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(png_path, img)

            transform = src.transform
            tile_width = src.width
            tile_height = src.height
            bounds = src.bounds

        annotations_written = False
        all_annotations = []

        # Check for both vector file types
        vector_files = [
            (os.path.join(vector_tiles_dir, f"{tile_id}_builds_vectors.shp"), 'builds'),  # Buildings (SHP)
            (os.path.join(vector_tiles_dir, f"{tile_id}_street_vectors.geojson"), 'street')  # Streets (GeoJSON)
        ]

        for vector_path, class_name in vector_files:
            if os.path.exists(vector_path):
                try:
                    # Read file ignoring problematic fields
                    gdf = gpd.read_file(vector_path)
                    
                    if not gdf.empty:
                        with rasterio.open(tif_path) as src:
                            if gdf.crs != src.crs:
                                gdf = gdf.to_crs(src.crs)

                        for _, row in gdf.iterrows():
                            class_id = class_map.get(class_name, 0)
                            geom = row.geometry
                            
                            # Skip invalid geometries
                            if geom is None or geom.is_empty or not geom.is_valid:
                                continue
                            
                            # Convert all geometries to polygons
                            polygons = []
                            if geom.geom_type == 'Polygon':
                                polygons = [geom]
                            elif geom.geom_type == 'MultiPolygon':
                                polygons = list(geom.geoms)
                            elif geom.geom_type in ['LineString', 'MultiLineString']:
                                # Convert lines to buffered polygons (adjust buffer size as needed)
                                buffered = geom.buffer(1.0)  # Adjust buffer distance based on your data
                                if buffered.geom_type == 'Polygon':
                                    polygons = [buffered]
                                elif buffered.geom_type == 'MultiPolygon':
                                    polygons = list(buffered.geoms)
                            
                            for polygon in polygons:
                                # Get exterior coordinates
                                if polygon.geom_type == 'Polygon':
                                    exterior = polygon.exterior
                                    coords = list(exterior.coords)
                                    
                                    # Convert to pixel coordinates
                                    pixel_coords = []
                                    for x, y in coords:
                                        x_pix = (x - bounds.left) / (bounds.right - bounds.left) * tile_width
                                        y_pix = (bounds.top - y) / (bounds.top - bounds.bottom) * tile_height
                                        
                                        # Normalize to [0,1]
                                        x_norm = max(0, min(1, x_pix / tile_width))
                                        y_norm = max(0, min(1, y_pix / tile_height))
                                        pixel_coords.extend([x_norm, y_norm])
                                    
                                    # Skip if too few points
                                    if len(pixel_coords) >= 6:  # At least 3 points (x,y,x,y,x,y)
                                        annotation = f"{class_id} " + " ".join([f"{coord:.6f}" for coord in pixel_coords])
                                        all_annotations.append(annotation)
                                        annotations_written = True
                except Exception as e:
                    print(f"Error processing {vector_path}: {str(e)}")
                    continue

        # Write all annotations to file
        if annotations_written:
            with open(txt_path, 'w') as f:
                f.write('\n'.join(all_annotations))
        elif save_empty_labels:
            open(txt_path, 'w').close()

    print(f"Converted {len(tiff_files)} tiles to PNG and YOLO-Seg format.")

class_map = {
    'builds': 0,
    'street': 1
}

convert_tiles_to_png_and_yolo_seg(
    raster_tiles_dir=PROJECT_DIR + "/raster_tiles_Shymkent",
    vector_tiles_dir=PROJECT_DIR + "/vector_tiles_Shymkent",
    output_img_dir=PROJECT_DIR + "/yolo_images_Shymkent",
    output_lbl_dir=PROJECT_DIR + "/yolo_labels_Shymkent",
    class_map=class_map
)

### Converting vector tiles to segmentation masks for UNet - check

In [ ]:


def vectors_to_masks(raster_tiles_dir, vector_tiles_dir, output_dir, target_size=(512, 512)):
    """
    Convert vector tiles to segmentation masks in PNG format.
    
    Args:
        raster_tiles_dir: Directory containing original TIFF tiles and metadata
        vector_tiles_dir: Directory containing clipped vector tiles
        output_dir: Where to save the output PNG masks
        target_size: Expected output size of masks (should match your TIFF tiles)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Class definitions
    CLASSES = {
        'background': 0,
        'builds': 1,
        'street': 2
    }
    
    # Get all TIFF tiles to process
    tiff_files = [f for f in os.listdir(raster_tiles_dir) if f.endswith('.tif')]
    
    stats = {
        'total_tiles': 0,
        'tiles_with_buildings': 0,
        'tiles_with_roads': 0,
        'tiles_with_both': 0,
        'empty_tiles': 0
    }
    
    for tiff_file in tqdm(tiff_files, desc="Processing tiles"):
        tile_id = os.path.splitext(tiff_file)[0]
        mask_path = os.path.join(output_dir, f"{tile_id}_mask.png")
        
        # Skip if mask already exists
        if os.path.exists(mask_path):
            continue
            
        # Open the raster tile to get dimensions and transform
        with rasterio.open(os.path.join(raster_tiles_dir, tiff_file)) as src:
            transform = src.transform
            width = src.width
            height = src.height
            crs = src.crs
            
        # Initialize empty mask
        mask = np.zeros((height, width), dtype=np.uint8)
        
        # Check for buildings
        builds_path = os.path.join(vector_tiles_dir, f"{tile_id}_builds_vectors.shp")
        has_buildings = False
        if os.path.exists(builds_path):
            try:
                builds = gpd.read_file(builds_path)
                if not builds.empty:
                    # Reproject if needed
                    if builds.crs != crs:
                        builds = builds.to_crs(crs)
                    
                    # Rasterize buildings (class 1)
                    shapes = [(geom, CLASSES['builds']) for geom in builds.geometry]
                    buildings_mask = rasterize(
                        shapes,
                        out_shape=(height, width),
                        transform=transform,
                        fill=0,
                        all_touched=True,
                        dtype=np.uint8
                    )
                    mask = np.where(buildings_mask > 0, CLASSES['builds'], mask)
                    has_buildings = True
            except Exception as e:
                print(f"Error processing buildings for {tile_id}: {str(e)}")
        
        # Check for roads
        roads_path_shp = os.path.join(vector_tiles_dir, f"{tile_id}_street_vectors.shp")
        roads_path_geojson = os.path.join(vector_tiles_dir, f"{tile_id}_street_vectors.geojson")
        has_roads = False
        
        # Try both shapefile and GeoJSON formats
        roads_path = roads_path_geojson if os.path.exists(roads_path_geojson) else roads_path_shp
        if os.path.exists(roads_path):
            try:
                roads = gpd.read_file(roads_path)
                if not roads.empty:
                    # Reproject if needed
                    if roads.crs != crs:
                        roads = roads.to_crs(crs)
                    
                    # Rasterize roads (class 2)
                    shapes = [(geom, CLASSES['street']) for geom in roads.geometry]
                    roads_mask = rasterize(
                        shapes,
                        out_shape=(height, width),
                        transform=transform,
                        fill=0,
                        all_touched=True,
                        dtype=np.uint8
                    )
                    # Only set road pixels where there isn't already a building
                    mask = np.where((roads_mask > 0) & (mask == 0), CLASSES['street'], mask)
                    has_roads = True
            except Exception as e:
                print(f"Error processing roads for {tile_id}: {str(e)}")
        
        # Update statistics
        stats['total_tiles'] += 1
        if has_buildings and has_roads:
            stats['tiles_with_both'] += 1
        elif has_buildings:
            stats['tiles_with_buildings'] += 1
        elif has_roads:
            stats['tiles_with_roads'] += 1
        else:
            stats['empty_tiles'] += 1
        
        # Save mask as PNG
        cv2.imwrite(mask_path, mask)
    
    # Print statistics
    print("\nMask Generation Statistics:")
    print(f"Total tiles processed: {stats['total_tiles']}")
    print(f"Tiles with buildings only: {stats['tiles_with_buildings']}")
    print(f"Tiles with roads only: {stats['tiles_with_roads']}")
    print(f"Tiles with both buildings and roads: {stats['tiles_with_both']}")
    print(f"Empty tiles: {stats['empty_tiles']}")
    
    return stats

# Example usage
mask_stats = vectors_to_masks(
    raster_tiles_dir=PROJECT_DIR+"//raster_tiles_Shymkent",
    vector_tiles_dir=PROJECT_DIR+"//vector_tiles_Shymkent",
    output_dir=PROJECT_DIR+"//segmentation_masks_Shymkent"
)

### Converting tiff tiles to png and vector tiles to TXT for YOLO OBB - check (improved the linestring buffer)

In [ ]:

def convert_tiles_to_png_and_yolo_obb(raster_tiles_dir, vector_tiles_dir, output_img_dir, output_lbl_dir, class_map, save_empty_labels=True):
    """
    Convert all raster tiles to PNG and generate YOLO-OBB annotations from both street (GeoJSON) and builds (SHP) vector data.
    Output format: <class-id> <x1> <y1> <x2> <y2> <x3> <y3> <x4> <y4> (normalized coordinates)
    """
    os.makedirs(output_lbl_dir, exist_ok=True)

    # Helper functions
    def world_to_pixel(geo_matrix, x, y, width, height):
        ulX = geo_matrix[0]
        ulY = geo_matrix[3]
        xDist = geo_matrix[1]
        yDist = geo_matrix[5]
        pixel = (x - ulX) / xDist
        line = (ulY - y) / -yDist
        return (pixel, line)

    def compute_pca_obb(points):
        points = np.array(points)
        if len(points) < 3:
            return None  # Degenerate geometry, return None
        try:
            center = points.mean(axis=0)
            cov = np.cov(points - center, rowvar=False)
            eigvals, eigvecs = np.linalg.eigh(cov)

            # Handle cases where covariance matrix is singular (e.g., line-like geometry)
            if np.any(np.isclose(eigvals, 0)):
                minx, miny = np.min(points, axis=0)
                maxx, maxy = np.max(points, axis=0)
                return [
                    [minx, miny], [maxx, miny],
                    [maxx, maxy], [minx, maxy]
                ]
            order = np.argsort(eigvals)[::-1]
            eigvecs = eigvecs[:, order]

            rot_points = (points - center) @ eigvecs
            min_x, min_y = np.min(rot_points, axis=0)
            max_x, max_y = np.max(rot_points, axis=0)

            rect = np.array([
                [min_x, min_y],
                [max_x, min_y],
                [max_x, max_y],
                [min_x, max_y]
            ])

            obb = (rect @ eigvecs.T) + center
            return obb.tolist()
        except Exception as e:
            print(f"Error in PCA OBB: {e}")
            return None  # Fallback if something goes wrong

    # Process each tile
    tiff_files = [f for f in os.listdir(raster_tiles_dir) if f.endswith('.tif')]
    for tif_file in tqdm(tiff_files, desc="Processing tiles"):
        tile_id = tif_file.replace('.tif', '')
        tif_path = os.path.join(raster_tiles_dir, tif_file)
        txt_path = os.path.join(output_lbl_dir, f"{tile_id}.txt")

        with rasterio.open(tif_path) as src:
            transform = src.transform
            tile_width = src.width
            tile_height = src.height
            bounds = src.bounds

        all_annotations = []

        # Process each vector file
        vector_files = [
            (os.path.join(vector_tiles_dir, f"{tile_id}_builds_vectors.shp"), 'builds'),  # Buildings (SHP)
            (os.path.join(vector_tiles_dir, f"{tile_id}_street_vectors.geojson"), 'street')  # Streets (GeoJSON)
        ]

        for vector_path, class_name in vector_files:
            if os.path.exists(vector_path):
                try:
                    gdf = gpd.read_file(vector_path)
                    
                    if not gdf.empty:
                        with rasterio.open(tif_path) as src:
                            if gdf.crs != src.crs:
                                gdf = gdf.to_crs(src.crs)

                        for _, row in gdf.iterrows():
                            class_id = class_map.get(class_name, 0)
                            geom = row.geometry
                            
                            # Skip invalid geometries
                            if geom is None or geom.is_empty or not geom.is_valid:
                                continue
                            
                            # Handle different geometry types
                            if geom.geom_type == 'Polygon':
                                polygons = [geom]
                            elif geom.geom_type == 'MultiPolygon':
                                polygons = list(geom.geoms)
                            elif geom.geom_type in ['LineString', 'MultiLineString']:
                                # Handle LineStrings differently
                                polygons = [geom]
                            else:
                                continue

                            # Convert geometry to bounding box
                            for polygon in polygons:
                                if polygon.geom_type == 'Polygon' and len(polygon.exterior.coords) > 2:
                                    coords = list(polygon.exterior.coords)
                                    try:
                                        # Handle LineString and MultiLineString separately
                                        if isinstance(polygon, LineString):
                                            obb = compute_pca_obb(coords)
                                        else:
                                            # Compute OBB using the convex hull or PCA
                                            hull = ConvexHull(coords)
                                            hull_points = np.array(coords)[hull.vertices]
                                            obb = compute_pca_obb(hull_points)
                                        
                                        if obb:
                                            # Order corners clockwise and normalize
                                            ordered_corners = order_corners_clockwise(obb)
                                            normalized_coords = []
                                            valid = True
                                            for x, y in ordered_corners:
                                                try:
                                                    px, py = world_to_pixel(transform, x, y, tile_width, tile_height)
                                                    norm_x = max(0, min(1, px / tile_width))
                                                    norm_y = max(0, min(1, py / tile_height))
                                                    normalized_coords.extend([norm_x, norm_y])
                                                except:
                                                    valid = False
                                                    break
                                            
                                            if valid and len(normalized_coords) == 8:
                                                annotation = f"{class_id} " + " ".join([f"{coord:.6f}" for coord in normalized_coords])
                                                all_annotations.append(annotation)
                                    except Exception as e:
                                        print(f"Error processing geometry: {e}")
                                        continue
                except Exception as e:
                    print(f"Error processing {vector_path}: {e}")
                    continue

        # Write all annotations to file
        if all_annotations:
            with open(txt_path, 'w') as f:
                f.write('\n'.join(all_annotations))
        elif save_empty_labels:
            open(txt_path, 'w').close()

    print(f"Converted {len(tiff_files)} tiles to YOLO-OBB format.")



### Checking the results of converting

In [ ]:
def visualize_yolo_seg_on_png(png_path, txt_path, class_colors=None):
    """
    Visualize YOLO-Seg annotations on a PNG image.

    """
    # Load image using OpenCV (BGR format)
    img = cv2.imread(png_path)
    if img is None:
        print(f"Could not load image: {png_path}")
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for plotting

    height, width = img.shape[:2]

    if not os.path.exists(txt_path):
        print(f"No annotation file found: {txt_path}")
        return

    with open(txt_path, 'r') as f:
        lines = f.readlines()

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img)

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7:
            continue  # Not enough coordinates for a polygon

        class_id = int(parts[0])
        coords = list(map(float, parts[1:]))

        if len(coords) % 2 != 0:
            continue

        # Denormalize coordinates
        points = []
        for i in range(0, len(coords), 2):
            x = coords[i] * width
            y = coords[i + 1] * height
            points.append((x, y))

        xs, ys = zip(*points)
        color = class_colors.get(class_id, 'red') if class_colors else 'red'

        # Draw polygon
        ax.plot(xs + (xs[0],), ys + (ys[0],), color=color, linewidth=1)
        ax.fill(xs, ys, alpha=0.2, color=color, label=f"Class {class_id}")

    ax.set_title(f"YOLO-Seg Annotations on {os.path.basename(png_path)}")
    plt.axis('on')
    plt.tight_layout()
    plt.show()

png_path = PROJECT_DIR + "//yolo_images_Shymkent//Shymkent_5cm_13_35_tile_0_1.png"
txt_path = PROJECT_DIR + "/yolo_labels_Shymkent//Shymkent_5cm_13_35_tile_0_1.txt"

class_colors = {
    0: 'blue',    # Buildings
    1: 'green'    # Roads
}

visualize_yolo_seg_on_png(png_path, txt_path, class_colors)
